In [105]:
import pandas as pd
import csv

# Step 1: Read the CSV and capture problematic lines
bad_lines = []
columns_count = 62  # The expected number of columns


# Function to capture bad lines that have more than 62 fields
def process_bad_line(line):
    fields = line.strip().split(",")  # Split by commas
    if len(fields) != columns_count:  # Check for field mismatch
        bad_lines.append(fields)


# Step 2: Read the file line by line and identify problematic rows
with open("../data/callsforservice.csv", "r") as file:
    for line in file:
        process_bad_line(line)


# Step 3: Write code to transform only the bad lines
def merge_delimited_values(row):
    merged_row = []
    skip_next = False
    long_hex_pattern = r"^[0-9A-F]{32,64}$"  # Regex for long hex pattern

    for i in range(len(row)):
        if skip_next:
            skip_next = False
            continue

        if re.match(r"D\d+", row[i]):
            if i + 2 < len(row) and re.match(r"\d+-\d+", row[i + 2]):
                # Merge the values between D-number and number-number pattern
                merged_value = ", ".join(row[i + 1 : i + 2])
                merged_row.append(row[i])
                merged_row.append(merged_value)
                merged_row.append(row[i + 2])
                skip_next = True
            elif i + 7 < len(row) and re.match(long_hex_pattern, row[i + 7]):
                # Merge the values between D-number and the long hex string
                merged_value = ", ".join(row[i + 1 : i + 7])
                merged_row.append(row[i])
                merged_row.append(merged_value)
                merged_row.append(row[i + 7])
                skip_next = True
            else:
                merged_row.append(row[i])
        else:
            if re.match(r"\{.*?\}", row[i]):
                cleaned_value = row[i].replace("'", "").replace('"', "")
                merged_row.append(cleaned_value)
            else:
                merged_row.append(row[i])

    return merged_row


# Step 4: Transform the bad lines
transformed_bad_lines = []
for line in bad_lines:
    cleaned_row = [field.replace("'", "").replace('"', "") for field in line]
    transformed_row = merge_delimited_values(cleaned_row)
    transformed_bad_lines.append(transformed_row)

# Step 5: Convert transformed bad lines to a DataFrame for further analysis
bad_lines_df = pd.DataFrame(transformed_bad_lines)

# Display the transformed bad lines
import ace_tools as tools

tools.display_dataframe_to_user(
    name="Transformed Bad Lines", dataframe=bad_lines_df
)

{Hamre Andrea}


In [112]:
import pandas as pd
import csv
import re

# Define the regex for the long hexadecimal-like string
long_hex_pattern = r"^[0-9A-F]{32,64}$"


def clean_curly_braces_content(field):
    # Regex to match content inside curly braces, removing quotes and commas within them
    return re.sub(
        r"\{([^{}]*)\}",
        lambda m: "{" + re.sub(r"[\"',]", "", m.group(1)) + "}",
        field,
    )


# Example usage in your merge_delimited_values function
def merge_delimited_values(row):
    merged_row = []
    skip_next = False
    long_hex_pattern = r"^[0-9A-F]{32,64}$"  # Regex for long hex pattern

    for i in range(len(row)):
        if skip_next:
            skip_next = False
            continue

        if re.match(r"D\d+", row[i]):
            if i + 2 < len(row) and re.match(r"\d+-\d+", row[i + 2]):
                # Merge the values between D-number and number-number pattern
                merged_value = ", ".join(row[i + 1 : i + 2])
                merged_row.append(row[i])
                merged_row.append(merged_value)
                merged_row.append(row[i + 2])
                skip_next = True
            elif i + 7 < len(row) and re.match(long_hex_pattern, row[i + 7]):
                # Merge the values between D-number and the long hex string
                merged_value = ", ".join(row[i + 1 : i + 7])
                merged_row.append(row[i])
                merged_row.append(merged_value)
                merged_row.append(row[i + 7])
                skip_next = True
            else:
                merged_row.append(row[i])
        else:
            if re.match(r"\{.*?\}", row[i]):
                cleaned_value = row[i].replace("'", "").replace('"', "")
                merged_row.append(cleaned_value)
            else:
                merged_row.append(row[i])

    return merged_row


# Function to handle the insertion of 14 commas after a closing curly bracket
def process_line_for_curly_bracket(line):
    # Check if the line ends with '}' followed by a new line
    if line.endswith("}\n") or line.endswith("}"):
        # Insert 14 commas at the end of the line
        line = line.strip() + "," * 14 + "\n"
    return line


# Variables to store good and bad rows
cleaned_lines = []  # For rows with exactly 62 columns
badrows = []  # For rows with more than 62 columns

# Open the file and clean the quotes before importing
with open("../data/callsforservice.csv", "r") as file:
    reader = csv.reader(file)

    for row in reader:
        # First, process each row to check for curly brackets followed by a new line
        row = [process_line_for_curly_bracket(field) for field in row]

        # Count the number of fields (columns) in the row
        if len(row) > 62:
            # Redirect rows with more than 62 columns to badrows
            badrows.append(row)
        else:
            # Remove stray quotes and merge delimited values based on the conditions
            cleaned_row = [
                field.replace("'", "").replace('"', "") for field in row
            ]
            merged_row = merge_delimited_values(cleaned_row)
            cleaned_lines.append(merged_row)

# Write the cleaned data to a temporary CSV file
with open(
    "../data/cleaned_callsforservice.csv", "w", newline=""
) as cleaned_file:
    writer = csv.writer(cleaned_file)
    writer.writerows(cleaned_lines)

# Convert bad rows into a DataFrame for further analysis
badrows_df = pd.DataFrame(badrows)

# Save bad rows to a new CSV file
badrows_df.to_csv("../data/badrows_callsforservice.csv", index=False)

# Now read the cleaned CSV file using pandas
callsforservice = pd.read_csv(
    "../data/cleaned_callsforservice.csv", on_bad_lines="warn"
)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

/tmp/ipykernel_992082/545254221.py:92: ParserWarning: Skipping line 331658: expected 62 fields, saw 63
Skipping line 333904: expected 62 fields, saw 63
Skipping line 333905: expected 62 fields, saw 63
Skipping line 334939: expected 62 fields, saw 63
Skipping line 334940: expected 62 fields, saw 63
Skipping line 334945: expected 62 fields, saw 63
Skipping line 334946: expected 62 fields, saw 63
Skipping line 335834: expected 62 fields, saw 63
Skipping line 335837: expected 62 fields, saw 63
Skipping line 335838: expected 62 fields, saw 63
Skipping line 335839: expected 62 fields, saw 63
Skipping line 335840: expected 62 fields, saw 63
Skipping line 335841: expected 62 fie

In [4]:
import pandas as pd

callsforservice = pd.read_csv("../data/callsforservice-quoted.csv", header=0)
callsforservice

/tmp/ipykernel_869863/2843026974.py:2: DtypeWarning: Columns (1,11,16,20,21,22,24,28,29,30,31,34,38,39,42,45,46,48,49,55,56,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  callsforservice = pd.read_csv('../data/callsforservice-quoted.csv', header=0)


,primarykey,agency,event_number,rev_num,call_datetime,call_type,call_desc,primary_unit,priority,call_source,...,call911_status,bluecam_status,wcs_status,oh1_status,countermeasures,victim,oshpoh1id,duplicate_of,is_motorcycle,ytids
0,8393467.0,CPD,202300352769,2.0,2023-11-22 11:17:55+00,INFW,INFO FOR A WARRANT PICK UP,NaN,4.0,ANI/ALI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8393469.0,CPD,202300352770,2.0,2023-11-22 11:20:46+00,TRUN,TROUBLE - UNKNOWN CAUSE,NaN,2.0,ANI/ALI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8393470.0,CPD,202300352771,2.0,2023-11-22 11:22:19+00,RFS,REC FILE SECTION EVENT- (RFS USE ONLY),NaN,5.0,Phone,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8393471.0,CPD,202300352772,8.0,2023-11-22 11:22:33+00,CWEL,CHECK WELFARE OF A PERSON,2A37,2.0,ANI/ALI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8393472.0,CPD,202300352773,9.0,2023-11-22 11:22:51+00,SPOT,SHOTSPOTTER ALERT,4A34,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709546,8393459.0,CPD,202300352764,7.0,2023-11-22 10:56:24+00,SPOT,SHOTSPOTTER ALERT,2A31,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1709547,8393460.0,CPD,202300352765,1.0,2023-11-22 11:01:18+00,ALMR,ALARM - RESIDENTIAL,NaN,3.0,1234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1709548,8393463.0,CPD,202300352766,16.0,2023-11-22 11:05:38+00,DVX,DOM VIOL ASLT/THREATS SUSP ON SCENE/IN AREA,4A36,1.0,ANI/ALI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1709549,8393464.0,CPD,202300352767,9.0,2023-11-22 11:07:28+00,DVX,DOM VIOL ASLT/THREATS SUSP ON SCENE/IN AREA,4B38,1.0,ANI/ALI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
callsforservice.columns

Index(['primarykey', 'agency', 'event_number', 'rev_num', 'call_datetime',
       'call_type', 'call_desc', 'primary_unit', 'priority', 'call_source',
       'cvaddress', 'cvaddress_zip', 'apt', 'zip', 'x_coord', 'y_coord',
       'current_flag', 'cvcalldate', 'cvcalldow', 'cvcalltime', 'cvlegend',
       'iwgeoname', 'status', 'score', 'side', 'x', 'y', 'cv_districts',
       'arrival_datetime', 'sub_call_type', 'nature', 'pd_zone', 'pd_district',
       'dispo_desc', 'cv_zones', 'cv_spa', 'dispo_desc_final', 'wkb_geometry',
       'update_datetime', 'event_status', 'pd_ward', 'cv_wards', 'cmc_case_no',
       'cmc_case_notes', 'local_report_no', 'narrative', 'assigned',
       'objectid', 'notes_misc', 'orig_call_desc', 'cad_notes_status',
       'call911_status', 'bluecam_status', 'wcs_status', 'oh1_status',
       'countermeasures', 'victim', 'oshpoh1id', 'duplicate_of',
       'is_motorcycle', 'ytids'],
      dtype='object')

In [15]:
# display all possible dispatch categories
# with pd.option_context('display.max_rows', None):
call_desc_counts = pd.DataFrame(
    callsforservice["is_motorcycle"]
    .value_counts()
    .sort_values(ascending=False)
)
print(call_desc_counts)

               count
is_motorcycle       
Y                 64


In [13]:
callsforservice["ytids"].value_counts().sort_values(ascending=False)

ytids
P87WS7ZWHAg                                                  1
Au2BtxzRrnk                                                  1
lmTA_s_2YDE                                                  1
ISUunBzeUz4                                                  1
5W8LpbbQAas                                                  1
KI39kjqK_9k                                                  1
6rLkNJ0Frd4                                                  1
be7bpV2ZuJ4                                                  1
ylLnLOEw0wQ                                                  1
ti7T8qFCI5U,wMxodj6sLAU                                      1
iko41z98NqQ,THxh4255rDM                                      1
HVxWxCRlJy0                                                  1
cSJ_EQMRXw8,rPJSPFoP7fg                                      1
Bint5c9epZE                                                  1
tg-L8aTCBYM                                                  1
G68HS1sNOjA,Lrc5pB61ItI                          

In [10]:
call_desc_counts["count"].sum()

1798

In [9]:
# Save bad rows to a new CSV file
call_desc_counts.to_csv("../data/narratives.csv")

https://bikepedcrashcle-tweepy.s3.amazonaws.com/reports/202100078250.pdf